In [1]:
import pyspark

In [2]:
# objeto principal o la base a partir de la cual cuelga toda la funcionalidad de Apache Spark
from pyspark.sql import SparkSession

# Se trata del context básico de Spark, desde donde se crean el resto de variables 
# que maneja el framework. Sólo un SparkContext puede estar activo por JVM.
from pyspark import SparkContext

# el objeto functions del modulo sql define las funciones estándar incorporadas 
# para trabajar con (valores producidos) columnas.
import pyspark.sql.functions as func

from datetime import datetime

In [3]:
spark = SparkSession.builder\
                    .appName("Test")\
                    .config("spark.jars.packages", "org.postgresql:postgresql:9.4.1211")\
                    .getOrCreate()

In [5]:
csv_file = 'allposts.csv'

In [6]:
df = spark.read.csv(csv_file, sep=',', escape='"', header=True, inferSchema=True, multiLine=True)

In [7]:
# change format in column created_utc
df = df.withColumn("created_utc",func.col("created_utc").cast("timestamp"))

In [8]:
# change format to column edited
df = df.withColumn('edited', 
               func.when(func.col('edited').like("1%"), 'True')
               .when(func.col('edited') == '', 'False')
               .otherwise(func.col('edited'))
              )
df = df.withColumn("edited",func.col("edited").cast("boolean"))

In [9]:
db_url = 'jdbc:postgresql://localhost:54321/postgres'
table_name = 'data_challenge2'
db_properties = {'user': 'postgres', 'password': 'postgres'}

In [12]:
%%time
df.write.option("numPartitions","5").option("batchsize","200000").jdbc(
            url=db_url,
            table=table_name,
            mode='overwrite',
            properties=db_properties)

CPU times: user 93.2 ms, sys: 63.4 ms, total: 157 ms
Wall time: 21min 59s
